In [881]:
import numpy as np 
import pandas as pd

In [882]:
movie=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [883]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [884]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### Merging the two dataset as per there title

In [886]:
movie.shape

(4803, 20)

In [887]:
credits.shape

(4803, 4)

In [888]:
movie=movie.merge(credits,on='title')
movie.shape

(4809, 23)

### processing or we can say Feature Engineering

In [890]:
# movie.head(1)
movie.columns
# movie.info()

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [891]:
(movie['id']==movie['movie_id']).value_counts() 

True     4803
False       6
Name: count, dtype: int64

In [892]:
#numerical data is not appropriate with our content recommendation
# things like  Genres popularity id keywords title overview cast and crew
movie=movie[['genres' ,'movie_id', 'keywords', 'title' , 'overview' ,'cast', 'crew']]

In [893]:
movie.head()

,genres,movie_id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


### atlast we want only id , name , tags

In [895]:
# 3 thing
# id to identify in web 
# name of movie to suggest 
# and the tags by which we going to pick it 

In [896]:
movie.isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [897]:
movie.dropna(inplace=True)

In [898]:
movie.isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    0
cast        0
crew        0
dtype: int64

In [ ]:
movie.duplicated().sum()

In [ ]:
import ast
def convertTags(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [901]:
movie['genres']=movie['genres'].apply(convertTags)

In [902]:
movie['keywords']=movie['keywords'].apply(convertTags)

In [903]:
def convertCast(obj):
    l=[]
    c=0
    for i in ast.literal_eval(obj):
        if c<5:
            l.append(i['name'])
            c+=1
        else:
            break
    return l


In [904]:
movie['cast']=movie['cast'].apply(convertCast)

In [905]:
def convertDirector(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [906]:
movie['crew']=movie['crew'].apply(convertDirector)

In [907]:
movie.head()

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [908]:
movie['overview']=movie['overview'].apply(lambda x:x.split())

In [909]:
movie.head(1)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [910]:
# making them a single word sam worthington to samwortington

In [911]:
movie['genres']=movie['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movie['overview']=movie['overview'].apply(lambda x:[i.replace(' ','') for i in x])
movie['cast']=movie['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movie['crew']=movie['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [912]:
movie.head(5)

,genres,movie_id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,"[Action, Adventure, ScienceFiction]",49529,"[based on novel, mars, medallion, space travel...",John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [913]:
movie['tags']=movie['genres']+movie['overview']+movie['cast']+movie['crew']

In [914]:
df=movie[['movie_id','title','tags']]
df

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, John, Cart..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, El, Mariachi, just, ..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple's, honey..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TVMovie, ""Signed,, Se..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [915]:
df['tags']=df['tags'].apply(lambda x:" ".join(x).lower())

C:\Users\satya\AppData\Local\Temp\ipykernel_43548\4056143573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']=df['tags'].apply(lambda x:" ".join(x).lower())


In [916]:
df['tags'][0]

'action adventure fantasy sciencefiction in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez jamescameron'

### test score for our data now to vectorize our data 

In [918]:
# form is movie id , name , tags up to 5000 
# converting tags to vectors or 2d space which is coordinates  of a vectors
# closest vector to make the the recommendation 

In [919]:
# checking the frequency of every occuring word from the large word set without stop words 

In [920]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [921]:
vectors=cv.fit_transform(df['tags']).toarray()

In [922]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [923]:
cv.get_feature_names_out() 

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### streaming of the corpuss

In [925]:
# like loving love and loves can be in same steam steam 


In [926]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
# use to find root word of any 

In [927]:
def stem_tags(text):
    if not isinstance(text, str):
        return ""
    return " ".join([ps.stem(word) for word in text.split()])

In [928]:
df['tags'] = df['tags'].apply(stem_tags)

C:\Users\satya\AppData\Local\Temp\ipykernel_43548\3544289644.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem_tags)


In [929]:
df['tags'][0]

'action adventur fantasi sciencefict in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. samworthington zoesaldana sigourneyweav stephenlang michellerodriguez jamescameron'

In [930]:
from sklearn.metrics.pairwise import cosine_similarity


In [931]:
similarity=cosine_similarity(vectors)

In [1019]:
similarity

array([[1.        , 0.13636364, 0.0805823 , ..., 0.        , 0.        ,
        0.        ],
       [0.13636364, 1.        , 0.0805823 , ..., 0.03413944, 0.        ,
        0.        ],
       [0.0805823 , 0.0805823 , 1.        , ..., 0.03026138, 0.        ,
        0.        ],
       ...,
       [0.        , 0.03413944, 0.03026138, ..., 1.        , 0.08006408,
        0.0270666 ],
       [0.        , 0.        , 0.        , ..., 0.08006408, 1.        ,
        0.05634362],
       [0.        , 0.        , 0.        , ..., 0.0270666 , 0.05634362,
        1.        ]])

In [933]:
# when a movie given to us we want o find the index in our 

In [934]:
df['title'] =df['title'].apply(lambda x:( x.lower()))

C:\Users\satya\AppData\Local\Temp\ipykernel_43548\3810380333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] =df['title'].apply(lambda x:( x.lower()))


In [ ]:
     

def recommnendIndex(movie):
    movie = movie.lower()
    
    m_index = df[df['title'].str.lower() == movie].index[0]
    
    dis = similarity[m_index]
    mlist = sorted(list(enumerate(dis)), reverse=True, key=lambda x: x[1])[1:6]

    for i in mlist:
        print(df.iloc[i[0]].title)


In [1083]:
recommnendIndex('John Carter')

captain america: the first avenger
star trek: insurrection
the marine 4: moving target
flyboys
dune
